<a href="https://colab.research.google.com/github/KSohi-max/Sherlock_Holmes_NLP/blob/main/Final_Term_Project_3666_03_ANLP_ChatBot_Sherlock_Holmes_Trivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup Environment (Libraries)

In [ ]:
!pip install nltk==3.5

     |████████████████████████████████| 1.4 MB 4.2 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434694 sha256=df0f316cda35fa9c6bda33660def597bd16f757ec17ed2600e0a09ff0e3fa231
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import platform
cloud_computing = (platform.system()=="Linux") #if on Linux, it must be on the cloud
print("Current System:", platform.system(), "("+platform.machine()+")", "\nCloud Computing:", cloud_computing)
import psutil
print("CPU Count: ", psutil.cpu_count(logical=False), "/", psutil.cpu_count(), "("+platform.processor()+")")
print("Total Memory: ", psutil.virtual_memory().total >> 30, "GB")

Current System: Linux (x86_64) 
Cloud Computing: True
CPU Count:  2 / 4 (x86_64)
Total Memory:  25 GB


In [ ]:
%%time
from IPython.display import display
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

CPU times: user 22 s, sys: 8.06 s, total: 30.1 s
Wall time: 1min 58s


[nltk_data]    |   Unzipping misc/mwa_ppdb.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


# Get Data

In [ ]:
# download data
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!cp -r '/content/drive/My Drive/pickled_Sherlock_Holmes_corpus.zip' '/content'

In [ ]:
#source_dir='/content/drive/MyDrive/3666 - Applied Natural Language Processing/Term Project'
target_dir='/content'
!rm -rf $target_dir/pickled_Sherlock_Holmes_corpus/*
!cp -p "$source_dir"/pickled_Sherlock_Holmes_corpus.zip $target_dir
!unzip $target_dir/pickled_Sherlock_Holmes_corpus
# (Optional) Force rebuilding the Ball Tree model
!rm -rf $target_dir/*.pkl
# (Optional) Copy the picked Ball Tree model if the previously saved one is in good shape
# This saves time to rebuild the model from scratch
!cp -p "$source_dir"/*.pkl $target_dir

cp: missing destination file operand after '/pickled_Sherlock_Holmes_corpus.zip'
Try 'cp --help' for more information.
Archive:  /content/pickled_Sherlock_Holmes_corpus.zip
  inflating: pickled_Sherlock_Holmes_corpus/A_Case_of_Identity.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/A_Scandal_in_Bohemia.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/A_Study_In_Scarlet.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/Black_Peter.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/Charles_Augustus_Milverton.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/His_Last_Bow.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/Lady_Frances_Carfax.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/Shoscombe_Old_Place.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/Silver_Blaze.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/The_Abbey_Grange.pickle  
  inflating: pickled_Sherlock_Holmes_corpus/The_Adventure_of_the_Beryl_Coronet.pickle  
  inflating: pickled_Sherloc

In [ ]:
!ls -l $target_dir/pickled_Sherlock_Holmes_corpus | head -n 10

total 9240
-rw-r--r-- 1 root root 100594 Dec  4 14:38 A_Case_of_Identity.pickle
-rw-r--r-- 1 root root 122574 Dec  4 14:38 A_Scandal_in_Bohemia.pickle
-rw-r--r-- 1 root root 638824 Dec  4 14:38 A_Study_In_Scarlet.pickle
-rw-r--r-- 1 root root 114077 Dec  4 14:38 Black_Peter.pickle
-rw-r--r-- 1 root root  95704 Dec  4 14:38 Charles_Augustus_Milverton.pickle
-rw-r--r-- 1 root root  88957 Dec  4 14:38 His_Last_Bow.pickle
-rw-r--r-- 1 root root 109542 Dec  4 14:38 Lady_Frances_Carfax.pickle
-rw-r--r-- 1 root root  90796 Dec  4 14:38 Shoscombe_Old_Place.pickle
-rw-r--r-- 1 root root 134722 Dec  4 14:38 Silver_Blaze.pickle


# Dialog

In [ ]:
import abc
import re
from collections.abc import Sequence
from operator import itemgetter

class Dialog(abc.ABC):
    """
    A dialog listens for utterances, parses and interprets them, then updates
    its internal state. It can then formulate a response on demand.
    """

    def listen(self, text, response=True, **kwargs):
        """
        A text utterance is passed in and parsed. It is then passed to the
        interpret method to determine how to respond. If a response is
        requested, the respond method is used to generate a text response
        based on the most recent input and the current Dialog state.
        """
        # Parse the input
        sents = self.parse(text)

        # Interpret the input
        sents, confidence, kwargs = self.interpret(sents, **kwargs)

        # Determine the response
        if response:
            reply = self.respond(sents, confidence, **kwargs)
        else:
            reply = None

        # Return initiative
        return reply, confidence

    @abc.abstractmethod
    def parse(self, text):
        """
        Every dialog may need its own parsing strategy, some dialogs may need
        dependency vs. constituency parses, others may simply require regular
        expressions or chunkers.
        """
        return []

    @abc.abstractmethod
    def interpret(self, sents, **kwargs):
        """
        Interprets the utterance passed in as a list of parsed sentences,
        updates the internal state of the dialog, computes a confidence of the
        interpretation. May also return arguments specific to the response
        mechanism.
        """
        return sents, 0.0, kwargs

    @abc.abstractmethod
    def respond(self, sents, confidence, **kwargs):
        """
        Creates a response given the input utterances and the current state of
        the dialog, along with any arguments passed in from the listen or the
        interpret methods.
        """
        return None

# Define Text Normalizer

In [ ]:
import nltk
import unicodedata

from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin

from nltk.probability import FreqDist


class TextNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self, language='english', minimum=2, maximum=200):
        self.min = minimum
        self.max = maximum
        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords

    def normalize(self, document):
        return [
            self.lemmatize(token, tag).lower()
            for paragraph in document
            for sentence in paragraph
            for (token, tag) in sentence
            if token in list(self.reduced)
               and not self.is_punct(token) and not self.is_stopword(token)
        ]

    def lemmatize(self, token, pos_tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

    def fit(self, documents, y=None):
        return self

    def transform(self, documents):
        words = []
        docs = []
        for document in documents:
            docs.append(document)
            for para in document:
                for sent in para:
                    for token, tag in sent:
                        words.append(token)

        counts = FreqDist(words)
        self.reduced = set(
            w for w in words if counts[w] > self.min and counts[w] < self.max
        )

        return [
            ' '.join(self.normalize(doc)) for doc in docs
        ]

#  Ball Tree Algorithm (Nearest Neighbor)

In [ ]:
## Building the Nearest Neighbor model

import pickle
#from sklearn.externals import joblib
# Above import gives error: ImportError: cannot import name 'joblib' from 'sklearn.externals' (/usr/local/lib/python3.7/dist-packages/sklearn/externals/__init__.py)
# Changed to directly import joblib below, after consulting StackOverflow
import joblib
from sklearn.pipeline import Pipeline
from sklearn.neighbors import BallTree
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

class BallTreeRecommender(object):
    """
    Given input terms, provide k story recommendations
    """
    def __init__(self, k=3, **kwargs):
        self.k = k
        self.trans_path = "svd.pkl"
        self.tree_path = "tree.pkl"
        self.transformer = False
        self.tree = None
        self.load()

    def load(self):
        """
        Load a pickled transformer and tree from disk,
        if they exist.
        """
        if os.path.exists(self.trans_path):
            self.transformer = joblib.load(open(self.trans_path, 'rb'))
            self.tree = joblib.load(open(self.tree_path, 'rb'))
        else:
            self.transformer = False
            self.tree = None

    def save(self):
        """
        It takes a long time to fit, so just do it once!
        """
        joblib.dump(self.transformer, open(self.trans_path, 'wb'))
        joblib.dump(self.tree, open(self.tree_path, 'wb'))

    def fit_transform(self, documents):
        # Transformer will be False if pipeline hasn't been fit yet,
        # Trigger fit_transform and save the transformer and lexicon.
        # Initially from Module 08, TruncatedSVD(n_components=200) was specified.
        # Changed n_compoenents from 200 to 2, due to ValueError: n_components must be < n_features; got 200 >= 2
        if self.transformer == False:
            self.transformer = Pipeline([
                 ('norm', TextNormalizer(minimum=50, maximum=200)),
                #('norm', TextNormalizer(minimum=2, maximum=5000)),
                ('transform', Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('svd', TruncatedSVD(n_components=200))
                ])
                 )
            ])
            self.lexicon = self.transformer.fit_transform(documents)
            self.tree = BallTree(self.lexicon)
            self.save()

    def query(self, terms):
        """
        Given input list of noun keywords,
        return the k closest matching stories.
        :param terms: list of strings
        :return: list of document indices of documents
        """
        vect_doc = self.transformer.named_steps['transform'].fit_transform(
             wordpunct_tokenize(terms) # input paramter "terms" is already tokenized
            #terms
        )
        dists, inds = self.tree.query(vect_doc, k=self.k)
        return inds[0]

# PickledCorpusReader

In [ ]:
import os
import nltk
import codecs
import pickle

from nltk import pos_tag
from nltk import sent_tokenize
from nltk import wordpunct_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.api import CategorizedCorpusReader

DOC_PATTERN = r'(?!\.)[\w\s\d\-\_]+\.txt'
PKL_PATTERN = r'(?!\.)[\w\s\d\-\_]+\.pickle'
CAT_PATTERN = r'([a-zA-Z_\s]+)/.*'

class PickledCorpusReader(CategorizedCorpusReader, CorpusReader):
    def __init__(self, root, fileids=PKL_PATTERN, **kwargs):
        """
        Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining arguments
        are passed to the ``CorpusReader`` constructor.
        """
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN

        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids)

    def resolve(self, fileids, categories):
        """
        Returns a list of fileids or categories depending on what is passed
        to each internal corpus reader function. This primarily bubbles up to
        the high level ``docs`` method, but is implemented here similar to
        the nltk ``CategorizedPlaintextCorpusReader``.
        """
        if fileids is not None and categories is not None:
            raise ValueError("Specify fileids or categories, not both")

        if categories is not None:
            return self.fileids(categories)
        return fileids

    def text(self, fileids=None, categories=None):
        """
        Returns the document loaded from a pickled object for every file in
        the corpus. Similar to the BaleenCorpusReader, this uses a generator
        to acheive memory safe iteration.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path, enc, fileid in self.abspaths(fileids, True, True):
            with open(path, 'rb') as f:
                yield pickle.load(f)

    def docs(self, fileids=None, categories=None):
        """
        Returns a generator of paragraphs where each paragraph is a list of
        sentences, which is in turn a list of (token, tag) tuples.
        """
        for doc in self.text(fileids, categories):
            yield doc['document']

    def titles(self, fileids=None, categories=None):
        """
        Uses BeautifulSoup to identify titles from the
        head tags within the HTML
        """
        for doc in self.text(fileids, categories):
            yield doc['title']

    def paras(self, fileids=None, categories=None):
        """
        Returns a generator of paragraphs where each paragraph is a list of
        sentences, which is in turn a list of (token, tag) tuples.
        """
        for doc in self.docs(fileids, categories):
            for paragraph in doc:
                yield paragraph

    def sents(self, fileids=None, categories=None):
        """
        Returns a generator of sentences where each sentence is a list of
        (token, tag) tuples.
        """
        for paragraph in self.paras(fileids, categories):
            for sentence in paragraph:
                yield sentence

    def tagged(self, fileids=None, categories=None):
        for sent in self.sents(fileids, categories):
            for token in sent:
                yield token

    def words(self, fileids=None, categories=None):
        """
        Returns a generator of (token, tag) tuples.
        """
        for token in self.tagged(fileids, categories):
            yield token[0]

# Story Recommender 

In [ ]:
class StoryRecommender(Dialog):
    """ Story recommender dialog  """
    def __init__(self, story, recommender=BallTreeRecommender(k=3)):
        self.story = list(story.titles())
        self.recommender = recommender
        # Fit the recommender model with the corpus
        self.recommender.fit_transform(list(story.docs()))

    def parse(self, text):
        """ Extract ingredients from the text  """
        return pos_tag(wordpunct_tokenize(text))

    def interpret(self, sents, **kwargs):
        # If feedback detected, update the model
        if 'feedback' in kwargs:
            self.recommender.update(kwargs['feedback'])
        n_nouns = sum(1 for pos , tag in sents if pos.startswith("N"))
        confidence = n_nouns/len(sents)
        terms = [tag for pos, tag in sents if pos.startswith("N")]
        return terms, confidence, kwargs
      
    def respond(self, terms, confidence, **kwargs):
        """Returns a recommendation if the confidence is > 0.15 otherwise None.
        """
        if confidence < 0.15:
            return None
        output = ["Here are stories related to {}".format(", ".join(terms))]
        output += [
            "- {}".format(self.story[idx])
            for idx in self.recommender.query(terms)
        ]
        return "\n".join(output)

# Test Recommender 👉

In [ ]:
if __name__ == '__main__':
  corpus = PickledCorpusReader('pickled_Sherlock_Holmes_corpus')
  recommender = StoryRecommender(corpus)
  question = "Which stories contain door, child, woman, blood, money"
  print (recommender.listen(question))

(None, 0.0)


In [ ]:
#Testing corpus titles
print(len(list(corpus.titles())))
list(corpus.titles())[:5]

60


['A CASE OF IDENTITY',
 'A SCANDAL IN BOHEMIA',
 'A STUDY IN SCARLET',
 'THE ADVENTURE OF BLACK PETER',
 'THE ADVENTURE OF CHARLES AUGUSTUS MILVERTON']

In [ ]:
print(list(corpus.sents())[:5])

[[('A', 'DT'), ('CASE', 'NNP'), ('OF', 'NNP'), ('IDENTITY', 'NNP')], [('Arthur', 'NNP'), ('Conan', 'NNP'), ('Doyle', 'NNP')], [('"', 'VB'), ('My', 'PRP$'), ('dear', 'JJ'), ('fellow', 'JJ'), (',"', 'NN'), ('said', 'VBD'), ('Sherlock', 'NNP'), ('Holmes', 'NNP'), ('as', 'IN'), ('we', 'PRP'), ('sat', 'VBD'), ('on', 'IN'), ('either', 'DT'), ('side', 'NN'), ('of', 'IN'), ('the', 'DT'), ('fire', 'NN'), ('in', 'IN'), ('his', 'PRP$'), ('lodgings', 'NNS'), ('at', 'IN'), ('Baker', 'NNP'), ('Street', 'NNP'), (',', ','), ('"', 'NNP'), ('life', 'NN'), ('is', 'VBZ'), ('infinitely', 'RB'), ('stranger', 'JJ'), ('than', 'IN'), ('anything', 'NN'), ('which', 'WDT'), ('the', 'DT'), ('mind', 'NN'), ('of', 'IN'), ('man', 'NN'), ('could', 'MD'), ('invent', 'VB'), ('.', '.')], [('We', 'PRP'), ('would', 'MD'), ('not', 'RB'), ('dare', 'VB'), ('to', 'TO'), ('conceive', 'VB'), ('the', 'DT'), ('things', 'NNS'), ('which', 'WDT'), ('are', 'VBP'), ('really', 'RB'), ('mere', 'JJ'), ('commonplaces', 'NNS'), ('of', 'IN')

In [ ]:
print(recommender.parse(question))

[('Which', 'JJ'), ('stories', 'VBZ'), ('contain', 'NN'), ('door', 'NN'), (',', ','), ('child', 'NN'), (',', ','), ('woman', 'NN'), (',', ','), ('blood', 'NN'), (',', ','), ('money', 'NN')]


In [ ]:
print (recommender.listen(question))

(None, 0.0)


#To Do:


1.   Try to fix the problem of low quality returned answer.
2.   Try to return the paragraph(s) most relevant to the nouns specified in the question, together w/ the story's title.
